In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 3.9 MB/s 
     |████████████████████████████████| 6.6 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 3.1 MB/s 
     |████████████████████████████████| 141 kB 16.7 MB/s 
     |████████████████████████████████| 212 kB 47.1 MB/s 
     |████████████████████████████████| 115 kB 52.4 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully un

In [ ]:
def get_dataset(path):
    with open(path) as f:
        tokens = []
        ner_tags = []
        token = []
        ner_tag = []
        for line in f: 
            if line.strip('\n') == '':
                tokens.append(token)
                ner_tags.append(ner_tag)
                token = []
                ner_tag=[]
                continue
            t = line.strip('\n').split()[0]
            l = line.strip('\n').split()[-1]
            token.append(t)
            ner_tag.append(l)
    print(len(tokens), len(ner_tags))
    print(tokens[0], ner_tags[0])

    return {
        'tokens':tokens,
        'ner_tags':ner_tags
    }

raw_train_data = get_dataset('train.conll')
raw_dev_data = get_dataset('dev.conll')
raw_test_data = get_dataset('test.conll')


39930 39930
['August', '2018-Present'] ['O', 'O']
6845 6845
['BitTorrent'] ['O']
10268 10268
['Participated', 'in', 'iEMG', 'sensor', 'development', '.'] ['O', 'O', 'O', 'O', 'O', 'O']


# Mục mới

In [ ]:
from pprint import pprint
label_dict = {}
for i in raw_train_data['ner_tags']:
    for j in i:
        if j not in label_dict.keys():
            label_dict[j]=len(label_dict)


label_dict['I-email'] = len(label_dict)
pprint(label_dict)
reverse_label_dict = {v:k for k,v in label_dict.items()}


{'B-degree': 1,
 'B-email': 10,
 'B-job_position': 8,
 'B-name': 6,
 'B-organization': 3,
 'B-phone': 12,
 'B-skill': 5,
 'I-degree': 2,
 'I-email': 14,
 'I-job_position': 9,
 'I-name': 7,
 'I-organization': 4,
 'I-phone': 13,
 'I-skill': 11,
 'O': 0}


In [ ]:
import torch
class TrainData():
    def __init__(self,input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
    def __getitem__(self,idx):
        return [
            torch.tensor(self.input_ids[idx]),
            torch.tensor(self.attention_mask[idx]),
            torch.tensor(self.labels[idx])
        ]
    def __len__(self):
        return len(self.labels)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer('hola', add_special_tokens=True))
tokenizer('hola', add_special_tokens=False)

{'input_ids': [0, 739, 143, 2], 'attention_mask': [1, 1, 1, 1]}


{'input_ids': [739, 143], 'attention_mask': [1, 1]}

In [ ]:
ignore_index =-100
max_length=128
batch_size = 32

def pad_to_maxlength(l,max_length, pad_value):
    if len(l) < max_length:
        l+=[pad_value]*(max_length-len(l))
    else:
        l = l[:max_length]
    return l
def get_training_data(data, shuffle=True):
    # print(data.data['tokens'])
    tokens = []
    max_l = 0
    for i in data['tokens']:
        if i ==[]:
            print('empty')
        tokens.append(i)
        if len(i)> max_l:
            max_l = len(i)
    print(max_l)
    print('Checking')
    raw_labels = data['ner_tags']
    # for i in range(len(raw_labels)):
    #     for j in range(len(raw_labels[i])):
    #         try:
    #             raw_labels[i][j]= label_dict[raw_labels[i][j]]
    #         except:
    #             print(raw_labels[i][j])
    #             return
    raw_labels= [i for i in raw_labels if i!= []]
    labels = []

    tokens = [i for i in tokens if i!= []]
    input_ids = []
    attention_masks = []
    print(len(tokens),len(raw_labels))
    assert len(tokens) == len(raw_labels)
    for i,token in enumerate(tokens):
        
        tmp_input_ids = [tokenizer.cls_token_id]
        tmp_attention_mask = [1]
        tmp_labels = ['O']
        for token_index, t in enumerate(token):
            t_encoded = tokenizer(t, add_special_tokens=False)
            tmp_input_ids +=t_encoded['input_ids']
            tmp_label = [raw_labels[i][token_index]]*len(t_encoded['input_ids'])
            for label_idx, label_token in enumerate(tmp_label):
                if label_idx != 0 and label_token[0] == 'B':
                    label_token = 'I'+label_token[1:]
                    tmp_label[label_idx] = label_token
            tmp_labels += tmp_label
            tmp_attention_mask+= t_encoded['attention_mask']
        tmp_input_ids+=[tokenizer.sep_token_id]
        tmp_attention_mask += [1]
        tmp_labels += ['O']
        tmp_input_ids = pad_to_maxlength(tmp_input_ids, max_length,tokenizer.pad_token_id)
        tmp_attention_mask = pad_to_maxlength(tmp_attention_mask, max_length, 0)
        tmp_labels = [label_dict[i] for i in tmp_labels]
        tmp_labels = pad_to_maxlength(tmp_labels, max_length, ignore_index)


        input_ids.append(tmp_input_ids)
        attention_masks.append(tmp_attention_mask)
        labels.append(tmp_labels)
    print(input_ids[-1][:len(token)+10], labels[-1][:len(token)+10], sep ='\n')




 
   

    data_class= TrainData(
        input_ids,
        attention_masks,
        labels
    )


    loader = DataLoader(data_class, batch_size=batch_size,shuffle=shuffle )
    return loader

train_data = get_training_data(raw_train_data)
dev_data = get_training_data(raw_dev_data, shuffle=False)
test_data = get_training_data(raw_test_data, shuffle=False)

138
Checking
39930 39930
[0, 276, 35602, 49377, 5007, 32007, 48748, 11399, 6, 58745, 6, 79445, 6, 5, 75018, 6, 5, 2, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100]
104
Checking
6845 6845
[0, 9113, 6495, 22824, 16655, 2076, 350, 4293, 6, 4, 23687, 6, 4, 4602, 2, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100]
131
Checking
10268 10268
[0, 5024, 449, 59246, 6, 4, 3291, 4848, 34, 3291, 1463, 6, 4, 19032, 7, 32742, 123378, 6, 4, 94686, 42341, 6, 4, 14313, 214]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
!pip install seqeval


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 737 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=cb466d53285bea7eae85a97c58716221db0279e628279ce0813935710a3dde69
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
from transformers import AutoModelForTokenClassification
from seqeval.metrics import classification_report
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR

n_epochs=1
lr = 2e-5
log_step =10
eval_step =200
total_steps = n_epochs*len(train_data)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_dict))
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = LinearLR(optimizer,total_iters=total_steps)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:

## train_loop


if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'

model.to(device)
step =0
avg_loss = 0
best_f1 = 0
for ep in range(n_epochs):
    for i, batch in enumerate(train_data):
        model.train()
        
        input_ids, attention_mask,label =batch
  
        optimizer.zero_grad()
        out = model(input_ids.to(device), attention_mask.to(device)).logits
        out = out.permute(0,-1,1)
        # print(out.shape, label.shape)
        # print(out[0], label[0], attention_mask[0], input_ids[0])
        
        
        loss = loss_fn(out, label.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()
        avg_loss += loss.item()
        if step % eval_step ==0 or step == total_steps -1:
            if step == total_steps -1:
                model.load_state_dict(best_state_dict)
            model.eval()
            y_true = []
            y_pred = []
            with torch.no_grad():
                for ii, e_batch in enumerate(dev_data):
                
                    input_ids, attention_mask,labels = e_batch
                    for y in labels:
                        y = [reverse_label_dict[i.item()] for i in y if i != ignore_index]
                        y_true.append(y)
                    pred = model(input_ids.to(device), attention_mask.to(device)).logits
                    # pred = pred.permute(0,-1,1)
                    pred = torch.argmax(pred,dim=-1)
                    pred = pred.detach().cpu()
                    for idx,p in enumerate(pred):
                        p = [reverse_label_dict[p[i].item()] for i in range(len(p)) if input_ids[idx][i]!=tokenizer.pad_token_id]
                        y_pred.append(p)

            print(classification_report(y_true,y_pred, digits=4, zero_division=0))
            f1 = classification_report(y_true,y_pred,digits=4, zero_division=0, output_dict = True)['weighted avg']['f1-score']
            if f1 > best_f1:
                best_f1 = f1
                best_state_dict = model.state_dict()
        if step % log_step == 0:
            print('Step {}/{}, Loss {} Best F1: {}'.format(step,total_steps,avg_loss/log_step, best_f1))
            avg_loss = 0
    
        step+=1


    


              precision    recall  f1-score   support

      degree     0.0000    0.0000    0.0000       196
       email     0.0000    0.0000    0.0000        97
job_position     0.0000    0.0000    0.0000       357
        name     0.0000    0.0000    0.0000       102
organization     0.0000    0.0000    0.0000       577
       phone     0.0000    0.0000    0.0000        86
       skill     0.0000    0.0000    0.0000      1676

   micro avg     0.0000    0.0000    0.0000      3091
   macro avg     0.0000    0.0000    0.0000      3091
weighted avg     0.0000    0.0000    0.0000      3091

Step 0/1248, Loss 0.31024553775787356 Best F1: 0
Step 10/1248, Loss 2.8744718551635744 Best F1: 0
Step 20/1248, Loss 2.337919998168945 Best F1: 0
Step 30/1248, Loss 1.181889498233795 Best F1: 0
Step 40/1248, Loss 0.7593021273612977 Best F1: 0
Step 50/1248, Loss 0.6442781299352646 Best F1: 0
Step 60/1248, Loss 0.4411598235368729 Best F1: 0
Step 70/1248, Loss 0.547505646944046 Best F1: 0
Step 80/1248, 

KeyboardInterrupt: ignored

In [ ]:
# model.load_state_dict(best_state_dict)
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    tokens = []
    for ii, e_batch in enumerate(test_data):
       

        input_ids, attention_mask,labels = e_batch
        for i in input_ids:
            tmp = [jj for jj in i if jj !=1]
            # tmp = tmp[1:-1] ## bỏ cls, sep
            tokens.append(tmp)
        for y in labels:
            y = [reverse_label_dict[i.item()] for i in y if i != ignore_index]
            y_true.append(y)
        pred = model(input_ids.to(device), attention_mask.to(device)).logits
        # pred = pred.permute(0,-1,1)
        pred = torch.argmax(pred,dim=-1)
        pred = pred.detach().cpu()
        for idx,p in enumerate(pred):
            p = [reverse_label_dict[p[i].item()] for i in range(len(p)) if labels[idx][i]!=ignore_index]
            y_pred.append(p)

print(classification_report(y_true,y_pred, digits=4, zero_division=0))


### lưu lại

print(len(tokens),len(y_true),len(y_pred))

              precision    recall  f1-score   support

      degree     0.0000    0.0000    0.0000       270
       email     0.0000    0.0000    0.0000       163
job_position     0.0000    0.0000    0.0000       530
        name     0.0000    0.0000    0.0000       178
organization     0.0000    0.0000    0.0000       819
       phone     0.0000    0.0000    0.0000       144
       skill     0.0000    0.0000    0.0000      3005

   micro avg     0.0000    0.0000    0.0000      5109
   macro avg     0.0000    0.0000    0.0000      5109
weighted avg     0.0000    0.0000    0.0000      5109

10268 10268 10268


In [ ]:
for i in range(len(tokens)):
    if len(tokens[i])!= len(y_pred[i]):
        print('Error on {}'.format(i))

with open('test_results.txt','w') as f:
    for i in range(len(tokens)):
        for t, lab, pred in zip(tokens[i],y_true[i],y_pred[i]):
            f.write('\t'.join([tokenizer.decode(t),lab,pred,'\n']))
            f.write('\n')
    

In [ ]:
## Examples 
tmp_text = 'Adam is highly skilled in computer engineering . He knows Java , Python and C++'
label = ['O','B-name','O','O','O','O','O','B-skill','I-skill','O','O','O','O','B-skill','O','O','B-skill','O','B-skill','I-skill','O']
print(len(tokenizer(tmp_text)['input_ids']), len(label))
print(label)
tokenized = tokenizer(tmp_text)['input_ids']
for i in range(len(tokenized)):
    print(tokenizer.decode(tokenized[i]), label[i])

21 21
['O', 'B-name', 'O', 'O', 'O', 'O', 'O', 'B-skill', 'I-skill', 'O', 'O', 'O', 'O', 'B-skill', 'O', 'O', 'B-skill', 'O', 'B-skill', 'I-skill', 'O']
<s> O
Adam B-name
is O
highly O
skille O
d O
in O
computer B-skill
engineering I-skill
 O
. O
He O
knows O
Java B-skill
 O
, O
Python B-skill
and O
C B-skill
++ I-skill
</s> O


In [ ]:
def get_entity(tokens, labels):
    i = 0
    tag_list = []
    found = False
    tmp_token_list = []
    tmp_label = 'O'
    while (i < len(tokens)):
        # print(i, tokens[i], labels[i])

        
        if labels[i].startswith('B') and not found:
            tmp_token_list.append(tokens[i])
            tmp_label = labels[i][2:]
            found = True
            i+=1
        if found:
            if labels[i] == 'O':
                tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))
                tmp_token_list = []
                # print(tag_list[-1])
                tmp_label='O'
                found = False
            elif labels[i].startswith('I') and labels[i][2:]==tmp_label:
                tmp_token_list.append(tokens[i])
            elif labels[i].startswith('B'):
                tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))
                tmp_token_list = []
                tmp_token_list.append(tokens[i])
                tmp_label = labels[i][2:]
                found = False
        i+=1
    if len(tmp_token_list) > 0:
        tag_list.append((tokenizer.decode(tmp_token_list), tmp_label))
    
    print(tag_list)
print(tmp_text)
get_entity(tokenized,label)


Adam is highly skilled in computer engineering . He knows Java , Python and C++
[('Adam', 'name'), ('computer engineering', 'skill'), ('Java', 'skill'), ('Python', 'skill'), ('C++', 'skill')]
